In [11]:
import numpy as np
from scipy import stats
import scipy.io
from matplotlib import pyplot as plt
import time
import pstats

In [12]:
def Gibbs_whole(G, Y, T):
    # Gibbs sampling for GCHMMs with inference and parameter estimation
    # G: non-symmetric social networks
    # Y: evidence, observed data
    # T: num. of iterations
    N, _, D = G.shape
    _, S, _ = Y.shape
    
    ## Initialization
    X = np.zeros((N,D+1))
    R = np.zeros((N,D))
    #hyperparameters
    ap=2; bp=2; aa=2; ba=5; ab=2; bb=5; ar=2; br=5; 
    a1=2; b1=2; a0=2; b0=5;
    xi = stats.beta.rvs(ap, bp, size=1)
    alpha = stats.beta.rvs(aa,ba,size=1)
    beta = stats.beta.rvs(ab,bb,size=1)
    gamma = stats.beta.rvs(ar,br,size=1)
    theta1 = stats.beta.rvs(a1,b1,size=(1,S)) 
    theta0 = stats.beta.rvs(a0,b0,size=(1,S)) 
    
    ##Iterative Sampling
    B = T/2 # Burn-in from Iteration B
    Xbi = X # Burn-in for X
    parabi = np.zeros((1,2*S+4)) # Burn-in for all parameters
    NPI = np.zeros((N,D)) # Num. of previous infection

    for t in range(T):
        # Update the initial X, root
        NPI[:,0] = NumPreInf(X[:,0],G[:,:,0]);
        p1 = xi*(gamma**np.array(X[:,1]==0)*(1-gamma)**np.array(X[:,1]))
        p0 = (1-xi)*(1-(1-alpha)*(1-beta)**NPI[:,0])**X[:,1]*((1-alpha)*(1-beta)**NPI[:,0])**(X[:,1]==0)
        p = p1 / (p0+p1)
        X[:,0] = 0+(np.random.rand(N,)<=p)
    
        # Update intermediate X
        for i in range(1,D):
            NPI[:,i-1] = NumPreInf(X[:,i-1],G[:,:,i-1])
            NPI[:,i] = NumPreInf(X[:,i],G[:,:,i])
            tmp1 = np.exp(Y[:,:,i-1] @ np.log(theta1.T))*np.exp((1-Y[:,:,i-1]) @ np.log(1-theta1.T))
            p1 = gamma**(X[:,i+1]==0)*(1-gamma)**(X[:,i-1]+X[:,i+1])*(1-(1-alpha)*(1-beta)**NPI[:,i-1])**(X[:,i-1]==0) * tmp1.reshape((N,))
            tmp0 = np.exp(Y[:,:,i-1] @ np.log(theta0.T))*np.exp((1-Y[:,:,i-1]) @ np.log(1-theta0.T))
            p0 = gamma**X[:,i-1]*(1-(1-alpha)*(1-beta)**NPI[:,i])**X[:,i+1]*(1-alpha)**((X[:,i-1]==0)+(X[:,i+1]==0))*(1-beta)**(NPI[:,i-1]*(X[:,i-1]==0)+NPI[:,i]*(X[:,i+1]==0))*tmp0.reshape((N,))
            p = p1 / (p0+p1)
            X[:,i] = 0+(np.random.rand(N,)<=p)
        
        # Updata last X
        NPI[:,D-1] = NumPreInf(X[:,D],G[:,:,D-1])
        tmp1 = np.exp(Y[:,:,D-1] @ np.log(theta1.T))* np.exp((1-Y[:,:,D-1]) @ np.log(1-theta1.T))
        p1 = (1-gamma)**X[:,D-1]*(1-(1-alpha)*(1-beta)**NPI[:,D-1])**(X[:,D-1]==0)*tmp1.reshape((N,))
        tmp0 = np.exp(Y[:,:,D-1] @ np.log(theta0.T))*np.exp((1-Y[:,:,D-1]) @ np.log(1-theta0.T))
        p0 = gamma**X[:,D-1]*((1-alpha)*(1-beta)**NPI[:,D-1])**(X[:,D-1]==0)*tmp0.reshape((N,))
        p = p1/(p0+p1)
        X[:,D] = 0+(np.random.rand(N,)<=p)
        
        # Update auxiliary variable R: prob p has various approximations
        # p = min(alpha./(1-(1-alpha)*(1-beta).^NPI),1);
        # p = min(0.5*alpha*(1+(1-beta).^NPI)./(1-(1-alpha)*(1-beta).^NPI),1);
        # p = min(alpha*(1-beta).^NPI./(1-(1-alpha)*(1-beta).^NPI),1); 
        p = alpha / (alpha+beta * NPI)
        tmp = 2 - (np.random.rand(N,D) <= p)
        R = (X[:,0:D]==0)*X[:,1:]*tmp
        
        # Update parameters
        xi = stats.beta.rvs(ap+sum(X[:,0]),bp+N-sum(X[:,0]),size=1)
        gamma = stats.beta.rvs(ar+np.sum(X[:,0:D]*(X[:,1:]==0)),br+np.sum(X[:,0:D]*X[:,1:]))
        alpha = stats.beta.rvs(aa+np.sum(R==1),ba+ np.sum((X[:,0:D]==0)*(X[:,1:]==0))+np.sum(R==2))
        beta = stats.beta.rvs(ab+np.sum(R>1),bb+np.sum(NPI*((X[:,0:D]==0)^(R>1))))
    
        temp = np.transpose(np.repeat(np.expand_dims(X[:,1:], axis=2),S, axis = 2), axes = [0, 2, 1])
        theta1 = stats.beta.rvs(a1+np.sum(Y*temp, axis=(0,2)), b1+ np.sum((1-Y)*temp, axis=(0,2)), size = S).reshape((1,S))
        theta0 = stats.beta.rvs(a0+np.sum(Y*(temp==0), axis=(0,2)), b0+ np.sum((1-Y)*(temp==0), axis=(0,2)), size = S).reshape((1,S))
        
        # Burn-in
        if t>B:
            Xbi = Xbi + X
            parabi = parabi + np.c_[xi,alpha,beta,gamma,theta1,theta0]
    # prediction
    Xpred = Xbi/(T-B)
    parapred = parabi/(T-B)
    return [Xpred, parapred]

def NumPreInf(Xt, Gt):
    return ((Gt + Gt.T) > 0) @ Xt

In [13]:
Y = scipy.io.loadmat('Y.mat')['Y']
X = scipy.io.loadmat('X.mat')['X']
G = scipy.io.loadmat('G.mat')['G']

In [14]:
missing_rate = 0
mask = stats.bernoulli.rvs(missing_rate, size = Y.shape)
Ymask = Y * mask
Ytrue = Y * (1 - mask)

In [15]:
def work_whole(G, Y, T = 500):
    Gibbs_whole(G, Y, T = 500)

In [16]:
%prun -q -D Gibbs_whole.prof Gibbs_whole(G, Y, T = 500)

 
*** Profile stats marshalled to file 'Gibbs_whole.prof'. 


In [17]:
p = pstats.Stats('work_big.prof')
p.sort_stats('time', 'cumulative').print_stats(10)
pass

Mon May  1 01:13:05 2017    work_big.prof

         524723 function calls in 11.558 seconds

   Ordered by: internal time, cumulative time
   List reduced from 64 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    6.767    6.767   11.558   11.558 <ipython-input-24-74a7d184eb53>:1(Gibbs_big)
   107000    3.266    0.000    3.266    0.000 <ipython-input-24-74a7d184eb53>:83(NumPreInf)
    13512    0.553    0.000    0.553    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    54500    0.270    0.000    0.270    0.000 {method 'rand' of 'mtrand.RandomState' objects}
      500    0.116    0.000    0.116    0.000 {method 'repeat' of 'numpy.ndarray' objects}
   110004    0.115    0.000    0.115    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     3006    0.054    0.000    0.416    0.000 /opt/conda/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:909(rvs)
     4000    0.040    0.000    0.053    0.000 /opt/con